In [1]:
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, TensorDataset
import torchattacks
from torchsummary import summary
from torchvision import models, datasets, transforms
import cv2
import numpy as np
from matplotlib import pyplot as plt
from tqdm import tqdm
import numpy as np


import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


import sys
from watermarkJPEG import *
from JPEG_layer import * # some utils.

In [2]:
transform = transforms.Compose(
        [
            #transforms.Resize(256),
            #transforms.CenterCrop(224),
            transforms.ToTensor(),
            #transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ]
    )

dataset = torchvision.datasets.ImageNet(root="./data/", split='val',
                             transform=transform
                                       )

test_loader = torch.utils.data.DataLoader(dataset, batch_size=1, shuffle=True)

In [3]:
class Interpolate(nn.Module):
    def __init__(self,size=(224,224), mode='bilinear') -> None:
        super(Interpolate, self).__init__()
        self.size = size
        self.mode = mode
    def forward(self, input: torch.Tensor) -> torch.Tensor:
        #input = torch.squeeze(input)
        #return torch.unsqueeze(F.interpolate(input, size=self.size, mode=self.mode), dim=0)
        return F.interpolate(input, size=self.size, mode=self.mode)

In [4]:
def modelIntegration(pretrained_model):
    class IntegratedModel(nn.Module):
        def __init__(self, pretrained_model):
            super(IntegratedModel, self).__init__()
            interpolate = Interpolate()
            self.Integrated_Model = nn.Sequential(
                interpolate,
                pretrained_model
            )

        def forward(self, x):
            x = self.Integrated_Model(x)
            return x
    return IntegratedModel(pretrained_model)

In [5]:
pretrained_model = models.resnet18(pretrained=True,  progress = True).to(device)
#_ = pretrained_model.eval()

In [6]:
IntegratedModel = modelIntegration(pretrained_model).to(device)
_=IntegratedModel.eval()

In [7]:
def recreate_image(im_as_var):
    """
        Recreates images from a torch variable, sort of reverse preprocessing

    Args:
        im_as_var (torch variable): Image to recreate

    returns:
        recreated_im (numpy arr): Recreated image in array
    """
    reverse_mean = [-0.485, -0.456, -0.406]
    reverse_std = [1/0.229, 1/0.224, 1/0.225]
    recreated_im = torch.clone(im_as_var)
    recreated_im = recreated_im.cpu().numpy()[0]
    for c in range(3):
        recreated_im[c] /= reverse_std[c]
        recreated_im[c] -= reverse_mean[c]
    recreated_im[recreated_im > 1] = 1
    recreated_im[recreated_im < 0] = 0
    #recreated_im = np.round(recreated_im * 255)
    recreated_im = recreated_im.transpose(1, 2, 0)
    #recreated_im = np.uint8(recreated_im).transpose(1, 2, 0)
    # Convert RBG to GBR
    #recreated_im = recreated_im[..., ::-1]
    return recreated_im

In [8]:
def recreate_image(im_as_var):
    recreated_im = torch.clone(im_as_var)
    recreated_im = recreated_im.cpu().numpy()[0]
    #recreated_im = np.round(recreated_im * 255)
    recreated_im = recreated_im.transpose(1, 2, 0)
    #recreated_im = np.uint8(recreated_im).transpose(1, 2, 0)
    # Convert RBG to GBR
    #recreated_im = recreated_im[..., ::-1]
    return recreated_im

In [9]:
attack = torchattacks.attacks.pgd.PGD(IntegratedModel,
                                      eps=8/255,
                                      alpha=1/255)

In [10]:
test_loader

In [11]:
correct_cnt = 0
block_cnt = 0
cum_turb = np.zeros((8,8))
for _ in tqdm((range(len(test_loader)))):
    data, target = next(iter(test_loader))
    data, target = data.to(device), target.to(device)
    #data.requires_grad = True
    output = IntegratedModel(data)
    init_pred = output.max(1, keepdim=True)[1]
    attack_img = attack(data, init_pred[0])
    # we assume that thh accuracy of the model is 1
    turb_noise = (attack_img - data).cpu().numpy()[0].transpose(1, 2, 0)
    #print(type(turb_noise),turb_noise.shape)
    img_YUV = cv2.cvtColor(turb_noise, cv2.COLOR_BGR2YUV)*255
    img_Y = img_YUV[:,:,0].copy()
    block_index = -1
    for i in range(0,int(np.floor(img_YUV.shape[0]/8)*8),8):
        for j in range(0,int(np.floor(img_YUV.shape[1]/8)*8),8):
            block_index += 1
            current_block = img_Y[i:i+8,j:j+8]
            block_dct = cv2.dct(current_block)
            cum_turb += np.abs(block_dct)
            block_cnt +=1

  0%|          | 0/50000 [00:00<?, ?it/s]/home/deponce/anaconda3/lib/python3.8/site-packages/torch/nn/functional.py:3609: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  warnings.warn(
/home/deponce/anaconda3/lib/python3.8/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)
100%|██████████| 50000/50000 [15:51:21<00:00,  1.14s/it]  


In [12]:
    """
    img_re = recreate_image(attack_img.detach())
    #print(torch.max(img_re))
    print(torch.sum(attack_img-data))
    print(init_pred,"-->",target)
    print("#--------------------------------#")
    print("orig")
    plt.imshow(recreate_image(data.detach()), interpolation='none')
    plt.xticks([])
    plt.yticks([])
    plt.show()
    print("v")
    print("|")
    print("v")
    print("noisy")
    plt.imshow(img_re, interpolation='none')
    plt.xticks([])
    plt.yticks([])
    plt.show()
    print("#--------------------------------#")
    """

'\nimg_re = recreate_image(attack_img.detach())\n#print(torch.max(img_re))\nprint(torch.sum(attack_img-data))\nprint(init_pred,"-->",target)\nprint("#--------------------------------#")\nprint("orig")\nplt.imshow(recreate_image(data.detach()), interpolation=\'none\')\nplt.xticks([])\nplt.yticks([])\nplt.show()\nprint("v")\nprint("|")\nprint("v")\nprint("noisy")\nplt.imshow(img_re, interpolation=\'none\')\nplt.xticks([])\nplt.yticks([])\nplt.show()\nprint("#--------------------------------#")\n'

In [13]:
%pwd

'/home/deponce/ECE699'

In [14]:
block_cnt

179216434

In [15]:
cum_turb

array([[1.58489075e+09, 1.10609953e+09, 9.55979093e+08, 7.87738279e+08,
        6.54474643e+08, 6.07578847e+08, 5.18174214e+08, 4.06497216e+08],
       [1.13166132e+09, 9.72885545e+08, 8.56975178e+08, 7.17166788e+08,
        6.01022010e+08, 5.55717840e+08, 4.73239469e+08, 3.77744007e+08],
       [9.80189560e+08, 8.72715544e+08, 7.70449479e+08, 6.46776977e+08,
        5.47900180e+08, 5.08917178e+08, 4.39761886e+08, 3.60948821e+08],
       [8.17493728e+08, 7.42801160e+08, 6.56981810e+08, 5.51158415e+08,
        4.77151132e+08, 4.50386912e+08, 3.99251676e+08, 3.41682253e+08],
       [6.59811478e+08, 6.14297205e+08, 5.47841972e+08, 4.70519627e+08,
        4.18888762e+08, 3.98790572e+08, 3.64427124e+08, 3.25322782e+08],
       [5.78814667e+08, 5.43329292e+08, 4.89483029e+08, 4.30282225e+08,
        3.89501514e+08, 3.70492958e+08, 3.44795328e+08, 3.16170288e+08],
       [5.18017447e+08, 4.79659358e+08, 4.39623497e+08, 3.94639627e+08,
        3.64480569e+08, 3.50057242e+08, 3.30069289e+08, 3.

In [16]:
def zigzag_flat(input8x8):
    assert input8x8.shape == (8,8)
    zigzag = np.array((        [[0,   1,   5,  6,   14,  15,  27,  28],
    [2,   4,   7,  13,  16,  26,  29,  42],
    [3,   8,  12,  17,  25,  30,  41,  43],
    [9,   11, 18,  24,  31,  40,  44,  53],
    [10,  19, 23,  32,  39,  45,  52,  54],
    [20,  22, 33,  38,  46,  51,  55,  60],
    [21,  34, 37,  47,  50,  56,  59,  61],
    [35,  36, 48,  49,  57,  58,  62,  63]]))
    output64 = np.zeros(64)
    for i in range(64):
        for j in range(8):
            for k in range(8):
                if zigzag[j,k] == i:
                    output64[i] = input8x8[j,k]
    return output64

In [17]:
zigzag_flat(cum_turb)

array([1.58489075e+09, 1.10609953e+09, 1.13166132e+09, 9.80189560e+08,
       9.72885545e+08, 9.55979093e+08, 7.87738279e+08, 8.56975178e+08,
       8.72715544e+08, 8.17493728e+08, 6.59811478e+08, 7.42801160e+08,
       7.70449479e+08, 7.17166788e+08, 6.54474643e+08, 6.07578847e+08,
       6.01022010e+08, 6.46776977e+08, 6.56981810e+08, 6.14297205e+08,
       5.78814667e+08, 5.18017447e+08, 5.43329292e+08, 5.47841972e+08,
       5.51158415e+08, 5.47900180e+08, 5.55717840e+08, 5.18174214e+08,
       4.06497216e+08, 4.73239469e+08, 5.08917178e+08, 4.77151132e+08,
       4.70519627e+08, 4.89483029e+08, 4.79659358e+08, 4.37318591e+08,
       4.07728562e+08, 4.39623497e+08, 4.30282225e+08, 4.18888762e+08,
       4.50386912e+08, 4.39761886e+08, 3.77744007e+08, 3.60948821e+08,
       3.99251676e+08, 3.98790572e+08, 3.89501514e+08, 3.94639627e+08,
       3.84062685e+08, 3.56686930e+08, 3.64480569e+08, 3.70492958e+08,
       3.64427124e+08, 3.41682253e+08, 3.25322782e+08, 3.44795328e+08,
      